
# Custom Support Chatbot

The objective of this project is to build a custom chatbot that answers questions based on documents.


![Alt text](image-1.png)

In [1]:
# import libraries

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate

/Users/maitemarzoratti/anaconda3/envs/langchain/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.0) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


### Retrieve information from urls

In [2]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

In [3]:
# use the selenium scraper to load the documents
loader = SeleniumURLLoader(urls=urls)
docs_not_splitted = loader.load()

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

Created a chunk of size 1226, which is longer than the specified 1000


In [4]:
# Compute embeddings and store in deeplake vector store

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "maitemarzoratti"
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


100%|██████████| 127/127 [00:07<00:00, 17.31it/s]
-

Dataset(path='hub://maitemarzoratti/langchain_course_customer_support', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (127, 1)      str     None   
 metadata     json      (127, 1)      str     None   
 embedding  embedding  (127, 1536)  float32   None   
    id        text      (127, 1)      str     None   


['25f38dbc-6ab0-11ee-8c31-6e005755cfe9',
 '25f38ea2-6ab0-11ee-8c31-6e005755cfe9',
 '25f38ed4-6ab0-11ee-8c31-6e005755cfe9',
 '25f38efc-6ab0-11ee-8c31-6e005755cfe9',
 '25f38f24-6ab0-11ee-8c31-6e005755cfe9',
 '25f38f4c-6ab0-11ee-8c31-6e005755cfe9',
 '25f38f74-6ab0-11ee-8c31-6e005755cfe9',
 '25f3900a-6ab0-11ee-8c31-6e005755cfe9',
 '25f3903c-6ab0-11ee-8c31-6e005755cfe9',
 '25f39064-6ab0-11ee-8c31-6e005755cfe9',
 '25f3908c-6ab0-11ee-8c31-6e005755cfe9',
 '25f390aa-6ab0-11ee-8c31-6e005755cfe9',
 '25f390c8-6ab0-11ee-8c31-6e005755cfe9',
 '25f390e6-6ab0-11ee-8c31-6e005755cfe9',
 '25f3910e-6ab0-11ee-8c31-6e005755cfe9',
 '25f3912c-6ab0-11ee-8c31-6e005755cfe9',
 '25f3914a-6ab0-11ee-8c31-6e005755cfe9',
 '25f39168-6ab0-11ee-8c31-6e005755cfe9',
 '25f39190-6ab0-11ee-8c31-6e005755cfe9',
 '25f391ae-6ab0-11ee-8c31-6e005755cfe9',
 '25f391cc-6ab0-11ee-8c31-6e005755cfe9',
 '25f391ea-6ab0-11ee-8c31-6e005755cfe9',
 '25f3921c-6ab0-11ee-8c31-6e005755cfe9',
 '25f3923a-6ab0-11ee-8c31-6e005755cfe9',
 '25f39258-6ab0-

In [5]:
# let's see the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Home  Tech  How to Check Disk Usage in Linux (4 Methods)

How to Check Disk Usage in Linux (4 Methods)

Beebom Staff

Last Updated: June 19, 2023 5:14 pm

There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.

Monitor Disk Usage in Linux (2023)

Table of Contents

Check Disk Space Using the df Command
		
Display Disk Usage in Human Readable FormatDisplay Disk Occupancy of a Particular Type

Check Disk Usage using the du Command
		
Display Disk Usage in Human Readable FormatDisplay Disk Usage for a Particular DirectoryCompare Disk Usage of Two Directories


### Build Chatbot

In [6]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

Run the pipeline with a user query, and generate the answer based on the retrieved documents.

In [7]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="text-davinci-003", temperature=0)
answer = llm(prompt_formatted)
print(answer)

 You can check disk usage in Linux using the df command or by using a GUI tool such as the GDU Disk Usage Analyzer or the Gnome Disks Tool. The df command is used to check the current disk usage and the available disk space in Linux. The syntax for the df command is: df <options> <file_system>. The options to use with the df command are: a, h, t, and x. To install the GDU Disk Usage Analyzer, use the command: sudo snap install gdu-disk-usage-analyzer. To install the Gnome Disks Tool, use the command: sudo apt-get -y install gnome-disk-utility.
